# A First SkLearn Model

- Based only on the numerical values
- Based on Differences
- Target BattleResult, transformed to [-1, 1] Range, 

In [2]:
import sklearn
import sys
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from kedro.extras.datasets.pandas import CSVDataSet

### Loading Data

In [3]:
BattleResults = catalog.load('Battle_Results_Train')

2020-05-22 08:45:35,506 - kedro.io.data_catalog - INFO - Loading data from `Battle_Results_Train` (CSVDataSet)...


In [4]:
BattleResults.head()

,Name_1,Level_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,Name_2,Level_2,HP_2,Attack_2,Defense_2,Sp_Atk_2,Sp_Def_2,Speed_2,Legendary_2,WeatherAndTime,BattleResult
0,Psyduck,8,68,65,59,82,63,69,False,Weedle,75,336,192,172,103,103,279,False,Unknown,-336
1,Rapidash,86,691,707,492,571,571,752,False,Zapdos,72,607,523,499,722,523,577,True,Sunshine,133
2,Dratini,93,636,698,484,538,538,538,False,Marowak,73,446,529,720,328,529,284,False,Windy,324
3,Horsea,6,41,47,85,85,29,72,False,Moltres,86,919,886,797,1088,750,797,True,Night,-919
4,Golbat,78,571,519,449,400,481,567,False,Kingler,96,637,1178,1049,456,456,682,False,Unknown,-560


### Data Pipeline

In [5]:
class AttributesDiff(BaseEstimator, TransformerMixin):
    def __init__(self): # no *args or **kargs
        pass
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        diff = X.iloc[:, 0] - X.iloc[:, 1]
        return np.expand_dims(diff.values, 1)

In [23]:
def RelativeResult(X):
    HP = X.apply(lambda row: row["HP_1"] if row["BattleResult"] > 0 else row["HP_2"], axis=1).values
    relativized = X["BattleResult"].values / HP
    return relativized

battle_labels = RelativeResult(BattleResults)

In [ ]:
full_pipeline = ColumnTransformer([
        ("LevelDiff", AttributesDiff(), ["Level_1", "Level_2"]),
        ("HPDiff", AttributesDiff(), ["HP_1", "HP_2"]),
        ("DefenseDiff", AttributesDiff(), ["Defense_1", "Defense_2"]),
        ("SpeedDiff", AttributesDiff(), ["Speed_1", "Speed_2"]),
        ("Sp_AtkDiff", AttributesDiff(), ["Sp_Atk_1", "Sp_Atk_2"]),
        ("Sp_DefDiff", AttributesDiff(), ["Sp_Def_1", "Sp_Def_2"]),
    ])
battle_data = full_pipeline.fit_transform(BattleResults); #battle_data

### Train

#### Linear Regression

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [27]:
lin_reg = LinearRegression()
lin_reg.fit(battle_data, battle_labels)

LinearRegression()

In [42]:
battle_predictions = lin_reg.predict(battle_data)
lin_mse = mean_squared_error(battle_labels, battle_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

0.4761618078459664

#### Decision Tree

In [43]:
tree_reg = DecisionTreeRegressor()

In [51]:
scores = cross_val_score(tree_reg, battle_data, battle_labels,
                         scoring="neg_mean_squared_error", cv=5)

In [52]:
tree_rmse_scores = np.sqrt(-scores)
tree_rmse_scores

array([0.41566111, 0.41332225, 0.41623414, 0.41462129, 0.41529759])

In [45]:
tree_reg.fit(battle_data[:-100000], battle_labels[:-100000])

DecisionTreeRegressor()

In [50]:
battle_predictions = tree_reg.predict(battle_data[-100000:])
tree_mse = mean_squared_error(battle_labels[-100000:], battle_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.4106550687719407

#### SVD